Specify the input and output directories here

In [25]:
image_dir  = '/home/arowa/work/porichoy/trdg-output-images/bangla_dataset_v1.1/augmented/images'
output_dir = '/home/arowa/work/porichoy/trdg-output-images/bangla_dataset_v1.1/augmented/aug_images'

Necessary Imports

In [26]:
import os
import random as rnd
import cv2
import numpy as np
from PIL import ImageEnhance
from PIL import Image
from tqdm import tqdm

# Augmentation Pipeline

### Function for Adding Noise to image

In [28]:
def noisy(image):
    '''Adds extra noise to image
    image (nd array) : input image
    '''
    types = ['gauss', 'poisson', 's&p']
    noise_type = rnd.choice(types)
    
    # adds gaussian noise
    if noise_type == "gauss":
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,image.shape)
        gauss = gauss.reshape(image.shape[0],image.shape[1]).astype('uint8')
        noisy = cv2.add(image,gauss)
        return noisy
    
    # adds salt and pepper noise
    elif noise_type == "s&p":
        noisy = image.copy()
        prob = 0.1
        black = 123
        white = 240            
        probs = np.random.random(noisy.shape[:2])
        noisy[probs < (prob / 2)] = black
        noisy[probs > 1 - (prob / 2)] = white
        return noisy

    # adds poisson noise
    elif noise_type == 'poisson':
        vals = len(np.unique(image))
        vals = 4 ** np.ceil(np.log2(vals))
        noisy = np.uint8(np.random.poisson(image * vals) / float(vals))
        return noisy

### Function for adding gradient effect to image

In [29]:
def gradient(image):
    '''Applies gradient in random direction
    image (nd array): input image
    '''
    # direction is specified here
    directions = [1, 2]
    direction = rnd.choice(directions)

    # gradient colour thresholds are specified here
    threshold1 = rnd.uniform(0.3, 1)
    threshold2 = rnd.uniform(0.3, 1)

    if direction == 1:
        mask = np.tile(np.linspace(threshold1, threshold2, image.shape[1]), (image.shape[0], 1))
    elif direction == 2:
        mask = np.tile(np.linspace(threshold1, threshold2, image.shape[0]), (image.shape[1], 1)).T
    gradient_image = np.uint8(image * mask)
    return gradient_image

### Function for Controlling brightness of image

In [30]:
def add_brightness(image):
    '''
    sets the brightness of the image to the desired value
    '''
    # image brightness enhancer
    value = rnd.uniform(0.4, 1.4)
    enhancer = ImageEnhance.Brightness(Image.fromarray(image))
    im_output = enhancer.enhance(value)
    return im_output

### Function to randomly crop image from sides

In [31]:
def get_random_crop(image):
    '''
    Randomly crops a portion of the image
    '''
    width = image.shape[1]
    height = image.shape[0]
    
    # if image is already small no need to crop it more
    if width <= 16 or height <= 32:
        return image

    crop_height_up = rnd.randint(0,8)
    crop_height_down = rnd.randint(0,8)
    crop_width_left = rnd.randint(0,7)
    crop_width_right = rnd.randint(0,7)
 
    crop = image[crop_height_up:height-crop_height_down,crop_width_left:width-crop_width_right]
    return crop

### Main Function

In [32]:
def main(in_path, out_path):

    # augmentation choices
    choices  = ['noisy', 'gradient', 'brightness']

    for filename in tqdm(os.listdir(in_path)):

        # we are using only once augmentation choice because multiple augmentations degrade the image too much
        # different augmentation techniques are given different percentage
        choice = rnd.choices(choices, weights=(85, 70, 50), k=1)

        filepath = os.path.join(in_path, filename)
        output_path = os.path.join(out_path, filename)

        image = cv2.imread(filepath,0)

        # we apply random crop augmentation to all images (no choice here)
        output_image = get_random_crop(image)
        # output_path = out_path + '/' +  filename.split('.')[0] + "_crop.jpg"
        # Image.fromarray(output_image).save(output_path)

        # augmentation according to choice
        if choice[0] == 'noisy':
            # output_path = out_path + '/' +  filename.split('.')[0] + "_noisy.jpg"
            output_image = noisy(output_image)
            Image.fromarray(output_image).save(output_path)

        elif choice[0] == 'gradient':
            # output_path = out_path + '/' +  filename.split('.')[0] + "_gradient.jpg"
            output_image = gradient(output_image)
            Image.fromarray(output_image).save(output_path)

        elif choice[0] == 'brightness':
            # output_path = out_path + '/' +  filename.split('.')[0] + "_brightness.jpg"
            output_image = add_brightness(output_image)
            output_image.save(output_path)

    return

### Driver call

In [33]:
main(image_dir, output_dir)

100%|██████████| 100000/100000 [01:22<00:00, 1210.94it/s]
